In [1]:
import mediapipe as mp
import cv2
import pickle 
import csv
import os
import numpy as np
import pandas as pd

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic 
mp_hands = mp.solutions.hands

In [16]:
mp_hands??

In [ ]:
cap = cv2.VideoCapture(0)
results1=0

with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
#     global results
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
       
        image.flags.writeable = False
        results1 = hands.process(image)
        #print(type(results1))


        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#         print(results.multi_hand_landmarks)
        #print(len(results1.multi_hand_landmarks))
        if results1.multi_hand_landmarks:
            landmarks=[]
            for hand_landmarks in results1.multi_hand_landmarks:
                #print(hand_landmarks.landmark)
                for lm in hand_landmarks.landmark:
                    landmarks.append(lm)
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            print(len(landmarks))
        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [36]:
num_coords = 21
#num_coords

In [37]:
land = ['class']
for val in range(1, num_coords+1):
    land += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [38]:
with open('hands.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(land)

In [67]:
class_name = "The “V” Sign"

In [68]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
#     global results
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue


        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)

        image.flags.writeable = False
        results = hands.process(image)


        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks=[]
            for hand_landmarks in results.multi_hand_landmarks:
                for lm in hand_landmarks.landmark:
                    #print()
                    landmarks.append(lm)
                #print(len(landmarks))
                hands_row = list(np.array([[mark.x, mark.y, mark.z] for mark in landmarks]).flatten())
                row = hands_row
                #print(len(row))
                row.insert(0, class_name)
                with open('hands.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(row)
                
                landmarks=[]
        
        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('hands.csv.csv')
X = df.drop('class', axis=1) # features
y = df['class'] # target value
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)


In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

fit_models['rc'].predict(X_test)

In [ ]:
df.tail()

In [66]:
df.shape

(954, 64)

In [48]:
import pandas as pd

In [47]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.22.2.post1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
C:\Users\KIIT\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator Pipeline from version 0.22.2.post1 when using version 0.24.1. This might lead to bre

In [49]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_hands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) as hands:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = hands.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            landmarks=[]
            for hand_landmarks in results.multi_hand_landmarks:
                for lm in hand_landmarks.landmark:
                    #print()
                    landmarks.append(lm)
                #print(len(landmarks))
                hands_row = list(np.array([[mark.x, mark.y, mark.z] for mark in landmarks]).flatten())
                row = hands_row
                X = pd.DataFrame([row])
                body_language_class = model.predict(X)[0]
                body_language_prob = model.predict_proba(X)[0]
                #print(body_language_class, body_language_prob)


                cv2.rectangle(image, 
                              (60, 100), 
                              (60+len(body_language_class)*20, 100-30), 
                              (245, 117, 16), -1)
                cv2.putText(image, body_language_class,(90,70) , 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Get status box
                cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)

                # Display Class
                cv2.putText(image, 'CLASS'
                            , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, body_language_class.split(' ')[0]
                            , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                # Display Probability
                cv2.putText(image, 'PROB'
                            , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                            , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                
                
                landmarks=[]
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

R [0.   0.04 0.01 0.   0.   0.   0.   0.01 0.01 0.05 0.21 0.01 0.07 0.
 0.02 0.01 0.   0.   0.   0.04 0.   0.01 0.02 0.   0.   0.04 0.38 0.
 0.   0.   0.   0.01 0.06 0.   0.  ]
B [0.   0.   0.01 0.   0.   0.   0.   0.   0.   0.   0.63 0.02 0.14 0.
 0.04 0.   0.   0.   0.   0.06 0.   0.   0.02 0.   0.   0.01 0.04 0.
 0.   0.   0.   0.01 0.01 0.   0.01]
D [0.07 0.   0.01 0.   0.   0.01 0.   0.02 0.   0.01 0.   0.   0.33 0.09
 0.03 0.   0.   0.   0.   0.   0.   0.   0.03 0.   0.03 0.01 0.03 0.01
 0.02 0.   0.01 0.01 0.   0.23 0.05]
D [0.02 0.   0.02 0.02 0.01 0.01 0.   0.01 0.01 0.   0.   0.   0.36 0.04
 0.01 0.   0.   0.   0.01 0.   0.   0.   0.03 0.   0.03 0.01 0.01 0.01
 0.02 0.   0.   0.   0.   0.28 0.09]
D [0.02 0.   0.   0.01 0.01 0.01 0.   0.04 0.   0.   0.   0.   0.39 0.01
 0.04 0.01 0.   0.   0.01 0.   0.   0.   0.03 0.   0.09 0.   0.01 0.01
 0.01 0.   0.   0.   0.   0.21 0.09]
D [0.01 0.   0.   0.01 0.01 0.   0.   0.03 0.   0.   0.   0.   0.35 0.05
 0.03 0.01 0.   0.   0.01 0.  

D [0.34 0.   0.01 0.   0.   0.01 0.   0.   0.01 0.01 0.01 0.   0.35 0.
 0.   0.   0.   0.04 0.   0.   0.   0.   0.   0.   0.   0.02 0.01 0.
 0.01 0.   0.   0.04 0.   0.13 0.01]
1 [0.39 0.   0.01 0.   0.   0.01 0.   0.   0.01 0.02 0.01 0.   0.26 0.
 0.   0.   0.   0.03 0.   0.   0.   0.   0.02 0.   0.   0.02 0.01 0.
 0.01 0.   0.   0.05 0.   0.14 0.01]
1 [0.4  0.   0.   0.   0.   0.02 0.   0.   0.   0.   0.02 0.01 0.25 0.01
 0.   0.   0.   0.03 0.   0.   0.   0.   0.   0.   0.   0.01 0.01 0.
 0.01 0.   0.   0.05 0.   0.16 0.02]
1 [0.43 0.   0.   0.   0.   0.02 0.   0.   0.   0.01 0.02 0.   0.22 0.02
 0.   0.   0.   0.03 0.   0.   0.   0.   0.   0.   0.   0.03 0.01 0.
 0.01 0.   0.   0.04 0.   0.15 0.01]
1 [0.45 0.   0.   0.   0.   0.02 0.   0.   0.   0.01 0.02 0.   0.2  0.01
 0.   0.   0.   0.03 0.   0.   0.   0.   0.   0.   0.   0.02 0.01 0.
 0.01 0.   0.   0.05 0.   0.16 0.01]
1 [0.44 0.   0.   0.   0.   0.02 0.   0.   0.   0.02 0.01 0.   0.23 0.01
 0.   0.   0.   0.03 0.   0.   0.   

R [0.1  0.   0.01 0.   0.   0.02 0.   0.04 0.   0.23 0.   0.   0.14 0.01
 0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.28 0.
 0.05 0.   0.   0.   0.   0.1  0.01]
A [0.19 0.   0.   0.   0.   0.04 0.   0.01 0.   0.39 0.   0.01 0.08 0.01
 0.01 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.2  0.
 0.01 0.   0.   0.   0.01 0.03 0.  ]
A [0.25 0.   0.   0.   0.   0.04 0.   0.01 0.   0.34 0.   0.   0.07 0.01
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.02 0.2  0.
 0.01 0.   0.   0.   0.01 0.04 0.  ]
A [0.31 0.   0.   0.   0.   0.03 0.   0.02 0.   0.33 0.   0.   0.06 0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.18 0.
 0.01 0.   0.   0.   0.01 0.03 0.01]
A [0.26 0.   0.   0.   0.   0.04 0.   0.02 0.   0.33 0.   0.   0.06 0.01
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.01 0.22 0.
 0.01 0.   0.   0.   0.01 0.03 0.  ]
A [0.28 0.   0.   0.   0.   0.01 0.   0.01 0.   0.34 0.   0.   0.07 0.01
 0.   0.   0.   0.   0.   0.   0. 

R [0.07 0.   0.01 0.   0.02 0.08 0.   0.04 0.   0.09 0.01 0.   0.07 0.01
 0.07 0.   0.   0.   0.01 0.   0.   0.   0.01 0.   0.   0.01 0.38 0.01
 0.01 0.   0.   0.01 0.   0.08 0.01]
R [0.06 0.   0.02 0.   0.01 0.09 0.   0.03 0.   0.09 0.01 0.   0.09 0.01
 0.07 0.   0.   0.   0.01 0.   0.   0.   0.04 0.   0.   0.   0.37 0.
 0.03 0.   0.   0.02 0.   0.05 0.  ]
R [0.08 0.   0.01 0.   0.02 0.09 0.   0.04 0.   0.11 0.01 0.   0.08 0.01
 0.06 0.   0.   0.   0.01 0.   0.   0.   0.03 0.   0.   0.   0.37 0.
 0.01 0.   0.   0.02 0.   0.05 0.  ]
R [0.14 0.   0.   0.   0.01 0.08 0.   0.03 0.02 0.17 0.   0.   0.1  0.
 0.06 0.   0.   0.   0.01 0.   0.   0.   0.03 0.   0.   0.   0.25 0.
 0.   0.   0.   0.02 0.   0.08 0.  ]
1 [0.22 0.   0.01 0.   0.   0.05 0.   0.04 0.   0.16 0.01 0.   0.12 0.
 0.02 0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.02 0.2  0.
 0.02 0.   0.   0.02 0.   0.09 0.01]
1 [0.22 0.   0.   0.   0.   0.06 0.   0.01 0.   0.05 0.   0.   0.14 0.01
 0.06 0.   0.   0.   0.03 0.01 0. 

A [0.08 0.   0.06 0.   0.02 0.   0.   0.01 0.02 0.35 0.02 0.   0.04 0.
 0.02 0.   0.   0.   0.   0.   0.   0.   0.09 0.   0.   0.06 0.1  0.01
 0.   0.   0.   0.08 0.   0.04 0.  ]
A [0.1  0.   0.04 0.   0.   0.01 0.   0.   0.01 0.28 0.12 0.01 0.18 0.
 0.02 0.   0.   0.   0.   0.01 0.   0.   0.01 0.   0.   0.02 0.06 0.01
 0.03 0.01 0.   0.04 0.01 0.02 0.01]
B [0.06 0.   0.02 0.   0.   0.   0.   0.   0.   0.03 0.38 0.   0.34 0.
 0.01 0.02 0.   0.   0.   0.06 0.   0.   0.02 0.   0.   0.01 0.01 0.
 0.01 0.   0.   0.02 0.01 0.   0.  ]
D [0.06 0.   0.   0.01 0.   0.   0.   0.   0.01 0.   0.12 0.02 0.27 0.03
 0.11 0.02 0.01 0.   0.   0.03 0.   0.   0.   0.   0.   0.02 0.   0.02
 0.04 0.01 0.   0.12 0.03 0.05 0.02]
D [0.07 0.   0.01 0.   0.01 0.   0.   0.   0.01 0.   0.05 0.03 0.18 0.01
 0.07 0.04 0.02 0.01 0.01 0.05 0.   0.   0.03 0.   0.   0.   0.   0.06
 0.01 0.01 0.   0.17 0.03 0.1  0.02]
D [0.02 0.   0.05 0.   0.   0.   0.   0.   0.   0.   0.06 0.   0.25 0.04
 0.   0.01 0.07 0.03 0.02 0.04

...

R [0.04 0.01 0.04 0.   0.13 0.   0.   0.02 0.   0.14 0.01 0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.01 0.   0.08 0.36 0.
 0.   0.   0.01 0.   0.02 0.   0.12]
R [0.05 0.01 0.03 0.   0.11 0.   0.   0.02 0.   0.17 0.   0.   0.01 0.
 0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.08 0.39 0.
 0.   0.   0.01 0.   0.02 0.   0.09]
R [0.05 0.   0.08 0.   0.09 0.01 0.   0.02 0.   0.25 0.   0.   0.01 0.
 0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.1  0.29 0.
 0.01 0.   0.01 0.   0.01 0.   0.06]
A [0.04 0.   0.08 0.01 0.06 0.   0.   0.03 0.   0.28 0.01 0.   0.02 0.
 0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.11 0.27 0.
 0.02 0.   0.01 0.01 0.01 0.01 0.02]
A [0.04 0.   0.04 0.02 0.02 0.   0.   0.03 0.   0.42 0.03 0.   0.04 0.
 0.   0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.1  0.16 0.01
 0.   0.   0.01 0.01 0.01 0.02 0.03]
A [0.03 0.   0.03 0.   0.02 0.   0.   0.   0.   0.51 0.05 0.   0.03 0.
 0.01 0.   0.   0.   0.   0.   0.   0.01 0

A [0.02 0.01 0.02 0.   0.   0.01 0.1  0.05 0.01 0.24 0.01 0.11 0.21 0.
 0.   0.   0.02 0.   0.   0.   0.03 0.02 0.01 0.   0.02 0.06 0.   0.
 0.   0.01 0.   0.01 0.   0.02 0.01]
A [0.03 0.   0.   0.   0.   0.   0.   0.01 0.01 0.71 0.05 0.   0.03 0.01
 0.03 0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.04 0.06 0.
 0.01 0.   0.   0.   0.   0.   0.  ]
D [0.02 0.01 0.02 0.   0.   0.   0.1  0.04 0.01 0.22 0.   0.13 0.26 0.
 0.   0.   0.01 0.   0.   0.   0.02 0.02 0.01 0.   0.02 0.05 0.   0.
 0.   0.01 0.   0.01 0.   0.02 0.02]
A [0.02 0.   0.   0.   0.   0.   0.   0.02 0.01 0.67 0.06 0.   0.05 0.01
 0.03 0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.   0.04 0.06 0.
 0.01 0.01 0.   0.   0.   0.   0.  ]
D [0.03 0.02 0.01 0.   0.   0.01 0.11 0.05 0.   0.22 0.01 0.1  0.24 0.
 0.   0.   0.01 0.   0.   0.   0.02 0.02 0.   0.   0.   0.06 0.01 0.
 0.   0.01 0.01 0.01 0.   0.03 0.02]
A [0.03 0.   0.   0.   0.   0.   0.   0.01 0.01 0.66 0.07 0.   0.06 0.01
 0.03 0.   0.   0.   0.   0.   0.   0.

A [0.05 0.   0.   0.   0.   0.01 0.   0.02 0.01 0.5  0.1  0.   0.08 0.01
 0.03 0.   0.   0.   0.   0.01 0.   0.02 0.   0.   0.   0.11 0.04 0.
 0.   0.01 0.   0.   0.   0.   0.  ]
A [0.02 0.03 0.01 0.   0.   0.   0.11 0.07 0.   0.24 0.04 0.11 0.19 0.
 0.   0.   0.01 0.   0.   0.   0.02 0.01 0.   0.   0.   0.07 0.01 0.
 0.   0.01 0.   0.01 0.   0.02 0.02]
A [0.05 0.   0.   0.01 0.   0.   0.   0.02 0.01 0.48 0.09 0.   0.08 0.01
 0.03 0.   0.   0.   0.   0.01 0.   0.03 0.   0.   0.   0.11 0.03 0.
 0.   0.01 0.   0.03 0.   0.   0.  ]
A [0.02 0.02 0.02 0.   0.   0.   0.11 0.08 0.   0.24 0.06 0.1  0.18 0.
 0.   0.   0.01 0.   0.   0.   0.03 0.   0.   0.   0.   0.07 0.   0.
 0.   0.01 0.   0.01 0.   0.02 0.02]
A [0.06 0.   0.   0.01 0.   0.01 0.   0.02 0.01 0.48 0.08 0.   0.09 0.01
 0.03 0.   0.   0.   0.   0.01 0.   0.03 0.   0.   0.   0.11 0.03 0.
 0.   0.01 0.   0.01 0.   0.   0.  ]
A [0.02 0.01 0.05 0.   0.   0.   0.09 0.05 0.   0.28 0.04 0.11 0.2  0.
 0.   0.   0.01 0.   0.   0.   0.02 0.

B [0.01 0.   0.01 0.   0.   0.   0.   0.   0.   0.03 0.28 0.   0.13 0.15
 0.06 0.01 0.03 0.   0.01 0.03 0.   0.   0.01 0.03 0.06 0.03 0.01 0.01
 0.02 0.01 0.   0.04 0.01 0.   0.02]
B [0.01 0.01 0.01 0.   0.   0.   0.   0.05 0.   0.01 0.46 0.   0.03 0.02
 0.   0.   0.   0.   0.05 0.   0.   0.   0.03 0.01 0.   0.02 0.01 0.
 0.01 0.   0.   0.06 0.01 0.   0.2 ]
B [0.01 0.   0.01 0.   0.   0.   0.   0.   0.   0.04 0.27 0.   0.12 0.19
 0.06 0.   0.03 0.   0.01 0.02 0.   0.01 0.01 0.   0.02 0.04 0.01 0.
 0.02 0.01 0.   0.06 0.02 0.   0.04]
B [0.01 0.02 0.01 0.   0.   0.   0.   0.06 0.   0.01 0.46 0.   0.03 0.02
 0.01 0.   0.   0.   0.05 0.   0.   0.   0.03 0.01 0.   0.02 0.01 0.
 0.01 0.   0.   0.07 0.   0.   0.17]
B [0.01 0.   0.01 0.   0.   0.   0.   0.   0.   0.04 0.27 0.   0.12 0.16
 0.06 0.   0.04 0.   0.01 0.02 0.   0.01 0.01 0.01 0.03 0.05 0.01 0.01
 0.02 0.01 0.   0.05 0.02 0.   0.03]
B [0.   0.01 0.   0.   0.   0.   0.   0.08 0.02 0.01 0.42 0.   0.02 0.02
 0.01 0.   0.   0.   0.04 0.

F [0.1  0.   0.   0.   0.01 0.   0.   0.02 0.01 0.   0.05 0.   0.17 0.02
 0.34 0.02 0.   0.   0.   0.   0.   0.   0.01 0.   0.   0.02 0.01 0.
 0.03 0.   0.   0.16 0.01 0.   0.02]


# Common Talk Signs

In [55]:
mp_holistic.HAND_CONNECTIONS??

In [7]:
cap = cv2.VideoCapture(0)
# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )
        if results.right_hand_landmarks is not None:
            print(len(results.right_hand_landmarks.landmark))

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21
21


In [5]:
print(len(results.pose_landmarks.landmark))

33


In [9]:
num_coords = len(results.pose_landmarks.landmark)+21+21 #pose+left+right
num_coords

75

In [12]:
landmarks = ['class']
for val in range(1, 34):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
for val in range(34, 77):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [13]:
landmarks

['class',
 'x1',
 'y1',
 'z1',
 'v1',
 'x2',
 'y2',
 'z2',
 'v2',
 'x3',
 'y3',
 'z3',
 'v3',
 'x4',
 'y4',
 'z4',
 'v4',
 'x5',
 'y5',
 'z5',
 'v5',
 'x6',
 'y6',
 'z6',
 'v6',
 'x7',
 'y7',
 'z7',
 'v7',
 'x8',
 'y8',
 'z8',
 'v8',
 'x9',
 'y9',
 'z9',
 'v9',
 'x10',
 'y10',
 'z10',
 'v10',
 'x11',
 'y11',
 'z11',
 'v11',
 'x12',
 'y12',
 'z12',
 'v12',
 'x13',
 'y13',
 'z13',
 'v13',
 'x14',
 'y14',
 'z14',
 'v14',
 'x15',
 'y15',
 'z15',
 'v15',
 'x16',
 'y16',
 'z16',
 'v16',
 'x17',
 'y17',
 'z17',
 'v17',
 'x18',
 'y18',
 'z18',
 'v18',
 'x19',
 'y19',
 'z19',
 'v19',
 'x20',
 'y20',
 'z20',
 'v20',
 'x21',
 'y21',
 'z21',
 'v21',
 'x22',
 'y22',
 'z22',
 'v22',
 'x23',
 'y23',
 'z23',
 'v23',
 'x24',
 'y24',
 'z24',
 'v24',
 'x25',
 'y25',
 'z25',
 'v25',
 'x26',
 'y26',
 'z26',
 'v26',
 'x27',
 'y27',
 'z27',
 'v27',
 'x28',
 'y28',
 'z28',
 'v28',
 'x29',
 'y29',
 'z29',
 'v29',
 'x30',
 'y30',
 'z30',
 'v30',
 'x31',
 'y31',
 'z31',
 'v31',
 'x32',
 'y32',
 'z32',
 'v32',
 '

In [14]:
with open('commontalks.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [15]:
class_name = "want to take tea"

In [30]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        

        results = holistic.process(image)

        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 

        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

        try:
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            #print("Pose",pose_row,"\n")
            lefthand = results.left_hand_landmarks.landmark
            lefthand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in lefthand]).flatten())
            #print("Left Hand",lefthand_row,"\n")
            righthand = results.right_hand_landmarks.landmark
            righthand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in righthand]).flatten())
            #print("Right Hand",righthand_row,"\n")
            
            

            row = pose_row + righthand_row + lefthand_row
            #print(row)
            
 
            row.insert(0, class_name)
            
            # Export to CSV
            with open('commontalks.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [31]:
df = pd.read_csv('commontalks.csv')

In [32]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z73,x74,y74,z74,x75,y75,z75,x76,y76,z76
0,want to take tea,0.549473,0.797456,-1.324226,1.0,0.588503,0.706161,-1.292615,1.0,0.617638,...,-0.204429,0.458067,0.874247,-0.207900,0.467399,0.873363,-0.210433,NaN,NaN,NaN
1,want to take tea,0.531372,0.721534,-0.832864,1.0,0.579667,0.652991,-0.802082,1.0,0.606936,...,-0.092725,0.364527,0.669604,-0.110848,0.387621,0.651914,-0.121597,NaN,NaN,NaN
2,want to take tea,0.619170,0.483134,-1.390251,1.0,0.660651,0.376522,-1.310694,1.0,0.689828,...,-0.097962,0.694711,0.765537,-0.097635,0.682517,0.760005,-0.096423,NaN,NaN,NaN


In [33]:
df.shape

(3, 262)